In [1]:
from DeepClassifier import DeepClassifier, create_classifier
from tensorflow.keras import optimizers
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

df = pd.read_csv('credit_risk_dataset.csv')

df

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,N,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,N,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,N,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,N,26


In [2]:
df["loan_status"].value_counts()

0    25473
1     7108
Name: loan_status, dtype: int64

In [3]:
print(f"{int(100*df[df['loan_status']==1].shape[0] / df.shape[0])}% of Rows are Defaults")

21% of Rows are Defaults


The dataset is imbalanced at almost 1:5 ratio, therefore the defaults are the least frequent class and the most important one.

Model performance will be assessed on Macro Averaged F1 Score, that shows robustness in this kind of scenario.

# Impute NaN and Drop Duplicates

In [4]:
df["person_emp_length"].fillna(df["person_emp_length"].median(), inplace=True)

df["loan_int_rate"].fillna(df["loan_int_rate"].median(), inplace=True)

df.drop_duplicates(inplace=True)

df[df.isnull().any(axis=1)]

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length


In [5]:
categorical_columns = df.columns[df.dtypes == 'object']

categorical_columns

Index(['person_home_ownership', 'loan_intent', 'loan_grade',
       'cb_person_default_on_file'],
      dtype='object')

In [6]:
for column in categorical_columns:
    df = pd.concat([df.drop(columns=[column]), pd.get_dummies(df[column], prefix=column, drop_first=True)],axis=1)

df

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_OTHER,person_home_ownership_OWN,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_Y
0,22,59000,123.0,35000,16.02,1,0.59,3,0,0,...,0,1,0,0,0,1,0,0,0,1
1,21,9600,5.0,1000,11.14,0,0.10,2,0,1,...,0,0,0,1,0,0,0,0,0,0
2,25,9600,1.0,5500,12.87,1,0.57,3,0,0,...,1,0,0,0,1,0,0,0,0,0
3,23,65500,4.0,35000,15.23,1,0.53,2,0,0,...,1,0,0,0,1,0,0,0,0,0
4,24,54400,8.0,35000,14.27,1,0.55,4,0,0,...,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,1.0,5800,13.16,0,0.11,30,0,0,...,0,1,0,0,1,0,0,0,0,0
32577,54,120000,4.0,17625,7.49,0,0.15,19,0,0,...,0,1,0,0,0,0,0,0,0,0
32578,65,76000,3.0,35000,10.99,1,0.46,28,0,0,...,0,0,0,1,0,0,0,0,0,0
32579,56,150000,5.0,15000,11.48,0,0.10,26,0,0,...,0,1,0,1,0,0,0,0,0,0


In [7]:
X = df.drop(columns='loan_status')
Y = df['loan_status']

scaler = StandardScaler().fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X.columns)

In [8]:
classifcator = DeepClassifier(
    model=create_classifier,
    loss="binary_crossentropy",
    model__architecture={
        "Layers": ["LSTM", "Dense"],
        "ActivationFunctions": ["relu", "sigmoid"],
        "Neurons": [50, 1],
    },
    callbacks=EarlyStopping,
    callbacks__monitor="val_loss",
    callbacks__min_delta=0.0001,
    callbacks__patience=20,
    callbacks__verbose=2,
    callbacks__restore_best_weights=True,
    verbose=2,
    epochs=100,
    train_ratio=0.8,
    val_ratio=0.2,
    batch_size_custom=32*8,
    fit__shuffle=True,
    optimizer=optimizers.Adam,
)


param_grid = {
    "optimizer__learning_rate": [0.00001, 0.0001, 0.001],
    "loss":["binary_crossentropy"],
    "model__architecture": [
        {
            "Layers": ["Dense", "Dense"],
            "ActivationFunctions": ["relu", "sigmoid"],
            "Neurons": [20, 1],
        },
        {
            "Layers": ["Dense", "Dense"],
            "ActivationFunctions": ["relu", "sigmoid"],
            "Neurons": [50, 1],
        },
        {
            "Layers": ["Dense", "Dense", "Dense"],
            "ActivationFunctions": ["relu", "relu", "sigmoid"],
            "Neurons": [50, 25, 1],
        },
        {
            "Layers": ["Dense", "Dense", "Dense", "Dense"],
            "ActivationFunctions": ["relu", "relu", "relu", "sigmoid"],
            "Neurons": [60, 30, 15, 1],
        },
    ],
}

clf = GridSearchCV(
    refit=True,
    estimator=classifcator,
    param_grid=param_grid,
    # Train all the models in paralel
    n_jobs=-1,
    cv=3,
    verbose=3,
    scoring='f1_macro',
)

# 80% train / 20% test
# Train set will be splitted again in order to use 20% of it for Keras validation

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))

clf.fit(X_train, y_train)

grid_search_cv = (
    pd.DataFrame.from_dict(clf.cv_results_)
    .sort_values(by=["rank_test_score"], ascending=True)
    .head(100)
)

grid_search_cv.index = grid_search_cv.rank_test_score
grid_search_cv.to_csv("GridSearchResults.csv")


pd.read_csv('GridSearchResults.csv')

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
55/55 - 1s - loss: 0.8291 - val_loss: 0.8169 - 1s/epoch - 24ms/step
Epoch 2/100
55/55 - 1s - loss: 0.7188 - val_loss: 0.7088 - 1s/epoch - 24ms/step
Epoch 2/100
55/55 - 1s - loss: 0.5765 - val_loss: 0.5710 - 1s/epoch - 24ms/step
Epoch 2/100
55/55 - 0s - loss: 0.8245 - val_loss: 0.8124 - 152ms/epoch - 3ms/step
Epoch 3/100
55/55 - 0s - loss: 0.7154 - val_loss: 0.7054 - 199ms/epoch - 4ms/step
Epoch 3/100
55/55 - 0s - loss: 0.5748 - val_loss: 0.5695 - 207ms/epoch - 4ms/step
Epoch 3/100
55/55 - 1s - loss: 0.7462 - val_loss: 0.7121 - 1s/epoch - 27ms/step
Epoch 2/100
55/55 - 1s - loss: 0.6680 - val_loss: 0.6354 - 1s/epoch - 26ms/step
Epoch 2/100
55/55 - 0s - loss: 0.8200 - val_loss: 0.8080 - 217ms/epoch - 4ms/step
Epoch 4/100
55/55 - 0s - loss: 0.7121 - val_loss: 0.7021 - 195ms/epoch - 4ms/st

55/55 - 0s - loss: 0.3339 - val_loss: 0.3263 - 203ms/epoch - 4ms/step
Epoch 9/100
55/55 - 0s - loss: 0.7822 - val_loss: 0.7708 - 251ms/epoch - 5ms/step
55/55 - 0s - loss: 0.9140 - val_loss: 0.8981 - 188ms/epoch - 3ms/step
Epoch 13/100
Epoch 9/100
55/55 - 0s - loss: 0.3362 - val_loss: 0.3332 - 208ms/epoch - 4ms/step
Epoch 9/100
55/55 - 0s - loss: 0.5158 - val_loss: 0.4932 - 226ms/epoch - 4ms/step
Epoch 11/100
55/55 - 0s - loss: 0.8008 - val_loss: 0.8032 - 170ms/epoch - 3ms/step
Epoch 8/100
55/55 - 0s - loss: 0.5596 - val_loss: 0.5549 - 182ms/epoch - 3ms/step
Epoch 13/100
55/55 - 0s - loss: 0.4909 - val_loss: 0.4711 - 218ms/epoch - 4ms/step
Epoch 10/100
55/55 - 0s - loss: 0.6836 - val_loss: 0.6734 - 233ms/epoch - 4ms/step
Epoch 13/100
55/55 - 0s - loss: 0.7023 - val_loss: 0.6963 - 163ms/epoch - 3ms/step
Epoch 8/100
55/55 - 0s - loss: 0.5084 - val_loss: 0.4894 - 195ms/epoch - 4ms/step
Epoch 12/100
55/55 - 0s - loss: 0.3294 - val_loss: 0.3235 - 161ms/epoch - 3ms/step
Epoch 10/100
55/55 - 0

55/55 - 0s - loss: 0.5482 - val_loss: 0.5440 - 198ms/epoch - 4ms/step
Epoch 21/100
55/55 - 0s - loss: 0.6629 - val_loss: 0.6576 - 213ms/epoch - 4ms/step
Epoch 16/100
55/55 - 0s - loss: 0.3041 - val_loss: 0.3103 - 198ms/epoch - 4ms/step
Epoch 17/100
55/55 - 0s - loss: 0.4307 - val_loss: 0.4174 - 236ms/epoch - 4ms/step
Epoch 20/100
55/55 - 0s - loss: 0.4377 - val_loss: 0.4221 - 178ms/epoch - 3ms/step
Epoch 20/100
55/55 - 0s - loss: 0.7472 - val_loss: 0.7363 - 216ms/epoch - 4ms/step
Epoch 22/100
55/55 - 0s - loss: 0.3049 - val_loss: 0.3068 - 161ms/epoch - 3ms/step
55/55 - 0s - loss: 0.3029 - val_loss: 0.3036 - 175ms/epoch - 3ms/step
Epoch 18/100
Epoch 18/100
55/55 - 0s - loss: 0.7459 - val_loss: 0.7491 - 188ms/epoch - 3ms/step
Epoch 17/100
55/55 - 0s - loss: 0.5468 - val_loss: 0.5427 - 166ms/epoch - 3ms/step
Epoch 22/100
55/55 - 0s - loss: 0.4193 - val_loss: 0.4083 - 223ms/epoch - 4ms/step
Epoch 19/100
55/55 - 0s - loss: 0.8556 - val_loss: 0.8404 - 248ms/epoch - 5ms/step
Epoch 18/100
55/5

55/55 - 0s - loss: 0.7185 - val_loss: 0.7079 - 172ms/epoch - 3ms/step
Epoch 30/100
55/55 - 0s - loss: 0.2887 - val_loss: 0.2898 - 219ms/epoch - 4ms/step
Epoch 26/100
55/55 - 0s - loss: 0.3975 - val_loss: 0.3867 - 197ms/epoch - 4ms/step
55/55 - 0s - loss: 0.2895 - val_loss: 0.2947 - 220ms/epoch - 4ms/step
Epoch 28/100
Epoch 26/100
55/55 - 0s - loss: 0.2897 - val_loss: 0.2993 - 239ms/epoch - 4ms/step
Epoch 25/100
55/55 - 0s - loss: 0.3896 - val_loss: 0.3826 - 212ms/epoch - 4ms/step
Epoch 27/100
55/55 - 0s - loss: 0.6366 - val_loss: 0.6260 - 191ms/epoch - 3ms/step
Epoch 30/100
55/55 - 0s - loss: 0.8090 - val_loss: 0.7942 - 209ms/epoch - 4ms/step
55/55 - 0s - loss: 0.7027 - val_loss: 0.7065 - 226ms/epoch - 4ms/step
Epoch 25/100
Epoch 26/100
55/55 - 0s - loss: 0.3869 - val_loss: 0.3768 - 204ms/epoch - 4ms/step
Epoch 29/100
55/55 - 0s - loss: 0.3937 - val_loss: 0.3835 - 160ms/epoch - 3ms/step
Epoch 29/100
55/55 - 0s - loss: 0.6243 - val_loss: 0.6198 - 223ms/epoch - 4ms/step
55/55 - 0s - loss

55/55 - 0s - loss: 0.7666 - val_loss: 0.7522 - 236ms/epoch - 4ms/step
Epoch 34/100
55/55 - 0s - loss: 0.3655 - val_loss: 0.3571 - 167ms/epoch - 3ms/step
Epoch 37/100
55/55 - 0s - loss: 0.6885 - val_loss: 0.6783 - 218ms/epoch - 4ms/step
Epoch 39/100
55/55 - 0s - loss: 0.2779 - val_loss: 0.2794 - 231ms/epoch - 4ms/step
Epoch 35/100
55/55 - 0s - loss: 0.2799 - val_loss: 0.2921 - 207ms/epoch - 4ms/step
Epoch 34/100
55/55 - 0s - loss: 0.5947 - val_loss: 0.5906 - 210ms/epoch - 4ms/step
55/55 - 0s - loss: 0.3704 - val_loss: 0.3631 - 212ms/epoch - 4ms/step
Epoch 33/100
Epoch 37/100
55/55 - 0s - loss: 0.3710 - val_loss: 0.3670 - 193ms/epoch - 4ms/step
Epoch 36/100
55/55 - 0s - loss: 0.2785 - val_loss: 0.2860 - 224ms/epoch - 4ms/step
Epoch 35/100
55/55 - 0s - loss: 0.6600 - val_loss: 0.6643 - 168ms/epoch - 3ms/step
Epoch 34/100
55/55 - 0s - loss: 0.6142 - val_loss: 0.6035 - 203ms/epoch - 4ms/step
Epoch 39/100
55/55 - 0s - loss: 0.7616 - val_loss: 0.7473 - 176ms/epoch - 3ms/step
Epoch 35/100
55/5

...

Epoch 63/100
55/55 - 0s - loss: 0.2716 - val_loss: 0.2763 - 229ms/epoch - 4ms/step
Epoch 65/100
55/55 - 0s - loss: 0.2736 - val_loss: 0.2856 - 229ms/epoch - 4ms/step
Epoch 64/100
55/55 - 0s - loss: 0.4058 - val_loss: 0.4004 - 232ms/epoch - 4ms/step
Epoch 71/100
55/55 - 0s - loss: 0.2088 - val_loss: 0.2580 - 211ms/epoch - 4ms/step
Epoch 74/100
55/55 - 0s - loss: 0.2074 - val_loss: 0.2491 - 208ms/epoch - 4ms/step
Epoch 74/100
55/55 - 0s - loss: 0.4260 - val_loss: 0.4164 - 178ms/epoch - 3ms/step
Epoch 72/100
55/55 - 0s - loss: 0.4808 - val_loss: 0.4871 - 193ms/epoch - 4ms/step
Epoch 68/100
55/55 - 0s - loss: 0.2711 - val_loss: 0.2761 - 197ms/epoch - 4ms/step
Epoch 66/100
55/55 - 0s - loss: 0.2663 - val_loss: 0.2717 - 208ms/epoch - 4ms/step
Epoch 64/100
55/55 - 0s - loss: 0.4036 - val_loss: 0.3982 - 192ms/epoch - 3ms/step
Epoch 72/100
55/55 - 0s - loss: 0.2729 - val_loss: 0.2847 - 208ms/epoch - 4ms/step
Epoch 65/100
55/55 - 0s - loss: 0.2099 - val_loss: 0.2603 - 207ms/epoch - 4ms/step
Epoc

55/55 - 0s - loss: 0.2658 - val_loss: 0.2800 - 199ms/epoch - 4ms/step
Epoch 76/100
55/55 - 0s - loss: 0.2603 - val_loss: 0.2673 - 203ms/epoch - 4ms/step
Epoch 75/100
55/55 - 0s - loss: 0.4099 - val_loss: 0.4010 - 197ms/epoch - 4ms/step
Epoch 84/100
55/55 - 0s - loss: 0.4488 - val_loss: 0.4551 - 184ms/epoch - 3ms/step
55/55 - 0s - loss: 0.2754 - val_loss: 0.2789 - 173ms/epoch - 3ms/step
Epoch 80/100
Epoch 10/100
55/55 - 0s - loss: 0.2617 - val_loss: 0.2695 - 201ms/epoch - 4ms/step
Epoch 78/100
55/55 - 0s - loss: 0.3807 - val_loss: 0.3761 - 198ms/epoch - 4ms/step
Epoch 84/100
55/55 - 0s - loss: 0.2009 - val_loss: 0.2511 - 204ms/epoch - 4ms/step
Epoch 87/100
55/55 - 0s - loss: 0.2597 - val_loss: 0.2670 - 206ms/epoch - 4ms/step
Epoch 76/100
55/55 - 0s - loss: 0.2655 - val_loss: 0.2795 - 217ms/epoch - 4ms/step
Epoch 77/100
55/55 - 0s - loss: 0.4087 - val_loss: 0.3999 - 209ms/epoch - 4ms/step
Epoch 85/100
55/55 - 0s - loss: 0.2725 - val_loss: 0.2788 - 166ms/epoch - 3ms/step
Epoch 11/100
55/5

55/55 - 0s - loss: 0.2535 - val_loss: 0.2644 - 197ms/epoch - 4ms/step
Epoch 91/100
55/55 - 0s - loss: 0.3935 - val_loss: 0.3856 - 168ms/epoch - 3ms/step
Epoch 98/100
55/55 - 0s - loss: 0.3630 - val_loss: 0.3589 - 159ms/epoch - 3ms/step
Epoch 97/100
55/55 - 0s - loss: 0.2584 - val_loss: 0.2736 - 170ms/epoch - 3ms/step
Epoch 90/100
55/55 - 0s - loss: 0.2342 - val_loss: 0.2591 - 172ms/epoch - 3ms/step
Epoch 24/100
55/55 - 0s - loss: 0.2537 - val_loss: 0.2629 - 179ms/epoch - 3ms/step
Epoch 89/100
55/55 - 0s - loss: 0.4177 - val_loss: 0.4232 - 175ms/epoch - 3ms/step
Epoch 94/100
55/55 - 0s - loss: 0.2530 - val_loss: 0.2641 - 185ms/epoch - 3ms/step
Epoch 92/100
55/55 - 0s - loss: 0.3925 - val_loss: 0.3846 - 193ms/epoch - 4ms/step
Epoch 99/100
55/55 - 0s - loss: 0.3619 - val_loss: 0.3578 - 184ms/epoch - 3ms/step
Epoch 98/100
55/55 - 0s - loss: 0.2578 - val_loss: 0.2733 - 175ms/epoch - 3ms/step
Epoch 91/100
55/55 - 0s - loss: 0.4158 - val_loss: 0.4213 - 162ms/epoch - 3ms/step
Epoch 95/100
55/5

82/82 - 0s - loss: 0.2333 - val_loss: 0.2374 - 109ms/epoch - 1ms/step
Epoch 17/100
82/82 - 0s - loss: 0.2310 - val_loss: 0.2366 - 103ms/epoch - 1ms/step
Epoch 18/100
82/82 - 0s - loss: 0.2283 - val_loss: 0.2362 - 100ms/epoch - 1ms/step
Epoch 19/100
82/82 - 0s - loss: 0.2280 - val_loss: 0.2314 - 98ms/epoch - 1ms/step
Epoch 20/100
82/82 - 0s - loss: 0.2254 - val_loss: 0.2319 - 100ms/epoch - 1ms/step
Epoch 21/100
82/82 - 0s - loss: 0.2231 - val_loss: 0.2339 - 100ms/epoch - 1ms/step
Epoch 22/100
82/82 - 0s - loss: 0.2228 - val_loss: 0.2440 - 95ms/epoch - 1ms/step
Epoch 23/100
82/82 - 0s - loss: 0.2227 - val_loss: 0.2306 - 99ms/epoch - 1ms/step
Epoch 24/100
82/82 - 0s - loss: 0.2217 - val_loss: 0.2317 - 99ms/epoch - 1ms/step
Epoch 25/100
82/82 - 0s - loss: 0.2187 - val_loss: 0.2315 - 104ms/epoch - 1ms/step
Epoch 26/100
82/82 - 0s - loss: 0.2181 - val_loss: 0.2327 - 96ms/epoch - 1ms/step
Epoch 27/100
82/82 - 0s - loss: 0.2184 - val_loss: 0.2319 - 97ms/epoch - 1ms/step
Epoch 28/100
82/82 - 0s

,rank_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_loss,param_model__architecture,param_optimizer__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score.1
0,1,14.199876,3.363520,0.312288,0.171500,binary_crossentropy,"{'Layers': ['Dense', 'Dense', 'Dense', 'Dense'...",0.00100,"{'loss': 'binary_crossentropy', 'model__archit...",0.861093,0.863231,0.864610,0.862978,0.001447,1
1,2,20.730299,1.522394,0.326097,0.016782,binary_crossentropy,"{'Layers': ['Dense', 'Dense', 'Dense'], 'Activ...",0.00100,"{'loss': 'binary_crossentropy', 'model__archit...",0.863187,0.859390,0.864675,0.862417,0.002225,2
2,3,23.633825,0.040451,0.258594,0.013647,binary_crossentropy,"{'Layers': ['Dense', 'Dense'], 'ActivationFunc...",0.00100,"{'loss': 'binary_crossentropy', 'model__archit...",0.858502,0.853509,0.859906,0.857306,0.002745,3
3,4,22.135793,0.144052,0.269236,0.025758,binary_crossentropy,"{'Layers': ['Dense', 'Dense'], 'ActivationFunc...",0.00100,"{'loss': 'binary_crossentropy', 'model__archit...",0.850417,0.846941,0.854477,0.850612,0.003080,4
4,5,24.747190,0.060657,0.124282,0.036397,binary_crossentropy,"{'Layers': ['Dense', 'Dense', 'Dense', 'Dense'...",0.00010,"{'loss': 'binary_crossentropy', 'model__archit...",0.851997,0.848097,0.848415,0.849503,0.001768,5
5,6,24.715788,0.089400,0.176915,0.013623,binary_crossentropy,"{'Layers': ['Dense', 'Dense', 'Dense'], 'Activ...",0.00010,"{'loss': 'binary_crossentropy', 'model__archit...",0.832280,0.837977,0.836496,0.835584,0.002414,6
6,7,29.904571,8.418953,0.247168,0.027549,binary_crossentropy,"{'Layers': ['Dense', 'Dense'], 'ActivationFunc...",0.00010,"{'loss': 'binary_crossentropy', 'model__archit...",0.821253,0.813774,0.825314,0.820114,0.004780,7
7,8,22.018293,0.038276,0.337561,0.007400,binary_crossentropy,"{'Layers': ['Dense', 'Dense'], 'ActivationFunc...",0.00010,"{'loss': 'binary_crossentropy', 'model__archit...",0.806838,0.797360,0.809699,0.804632,0.005273,8
8,9,24.560801,0.076935,0.203078,0.020352,binary_crossentropy,"{'Layers': ['Dense', 'Dense', 'Dense'], 'Activ...",0.00001,"{'loss': 'binary_crossentropy', 'model__archit...",0.723660,0.689562,0.729990,0.714404,0.017755,9
9,10,24.756986,0.103030,0.179066,0.026062,binary_crossentropy,"{'Layers': ['Dense', 'Dense', 'Dense', 'Dense'...",0.00001,"{'loss': 'binary_crossentropy', 'model__archit...",0.616854,0.758484,0.693446,0.689595,0.057884,10


In [9]:
X_test = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

y_pred = clf.best_estimator_.predict(X_test)

26/26 - 0s - 88ms/epoch - 3ms/step


In [10]:
predictions = [(value > 0.5).astype(int) for value in y_pred]

tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()

print(f"True Negative: {tn}\nFalse Positive: {fp}\nFalse Negative: {fn}\nTrue Positive: {tp}")

print("\n")

print(classification_report(y_test, y_pred, target_names = ['Non-Default', 'Default']))

True Negative: 4975
False Positive: 80
False Negative: 446
True Positive: 983


              precision    recall  f1-score   support

 Non-Default       0.92      0.98      0.95      5055
     Default       0.92      0.69      0.79      1429

    accuracy                           0.92      6484
   macro avg       0.92      0.84      0.87      6484
weighted avg       0.92      0.92      0.91      6484



In [11]:
print('Best Estimator')
print('-'*90)
print(clf.best_estimator_)
print('-'*90)

Best Estimator
------------------------------------------------------------------------------------------
DeepClassifier(
	model=<function create_classifier at 0x7feaf02823a0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=<class 'keras.optimizer_v2.adam.Adam'>
	loss=binary_crossentropy
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=2
	callbacks=<class 'keras.callbacks.EarlyStopping'>
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	model__architecture={'Layers': ['Dense', 'Dense', 'Dense', 'Dense'], 'ActivationFunctions': ['relu', 'relu', 'relu', 'sigmoid'], 'Neurons': [60, 30, 15, 1]}
	callbacks__monitor=val_loss
	callbacks__min_delta=0.0001
	callbacks__patience=20
	callbacks__verbose=2
	callbacks__restore_best_weights=True
	train_ratio=0.8
	val_ratio=0.2
	batch_size_custom=256
	fit__shuffle=True
	optimizer__learning_rate=0.001
	class_weight=None
)
---------------------------------------------------------------------------------